# Home Credit Default Risk

https://www.kaggle.com/c/home-credit-default-risk#description

https://www.kaggle.com/codename007/home-credit-complete-eda-feature-importance/notebook

* application_{train|test}.csv
 * main table
* bureau.csv
 * 其他機構之前的 credit
* bureau_balance.csv
 * bureau.csv 的每月餘額
* POS_CASH_balance.csv
* credit_card_balance.csv
* previous_application.csv
* installments_payments.csv
* HomeCredit_columns_description.csv

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [2]:
application_train = pd.read_csv('../input/application_train.csv')
#POS_CASH_balance = pd.read_csv('../input/POS_CASH_balance.csv')
#bureau_balance = pd.read_csv('../input/bureau_balance.csv')
#previous_application = pd.read_csv('../input/previous_application.csv')
#installments_payments = pd.read_csv('../input/installments_payments.csv')
#credit_card_balance = pd.read_csv('../input/credit_card_balance.csv')
#bureau = pd.read_csv('../input/bureau.csv')
#application_test = pd.read_csv('../input/application_test.csv')

## application_train

In [3]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## POS_CASH_balance data

In [4]:
POS_CASH_balance.head()

NameError: name 'POS_CASH_balance' is not defined

## bureau_balance data

In [5]:
bureau_balance.head()

NameError: name 'bureau_balance' is not defined

## previous_application data

In [6]:
previous_application.head()

NameError: name 'previous_application' is not defined

## installments_payments data

In [7]:
installments_payments.head()

NameError: name 'installments_payments' is not defined

## credit_card_balance data

In [8]:
credit_card_balance.head()

NameError: name 'credit_card_balance' is not defined

## bureau data

In [9]:
bureau.head()

NameError: name 'bureau' is not defined

# drop id, target

In [10]:

train_data = application_train.drop(['TARGET', 'SK_ID_CURR'], axis=1)
train_data.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## train_missing

In [11]:
#total = application_train.isnull().sum()#.sort_values(ascending = False)
train_missing = []
for var in train_data.columns:
    if train_data[var].isnull().sum() != 0:
        train_missing.append(var)
print(train_missing)

['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'OWN_CAR_AGE', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',

## application_discrete_features

In [37]:
cnt = 0

train_discrete_features = {}
for var in train_data.columns:
    #application_discrete_features[var] = 1.*application_train[var].nunique()/application_train[var].count() < 0.001 #or some other threshold
    train_discrete_features[var] = 1.*train_data[var].nunique() < 10

discrete_features = []
for f in train_data.columns:
    if train_discrete_features[f] == True:
        discrete_features.append(f)
        
print(discrete_features)
print(len(discrete_features))

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FL

## discrete to onehot

In [38]:

x_descrete = pd.DataFrame()

for f in train_data.columns:
    if train_discrete_features[f] == True:
        print(f)
        onehot = pd.get_dummies(train_data[f])
        onehot.columns = [f + '_' + str(col) for col in onehot.columns]
        x_descrete = pd.concat([x_descrete, onehot],axis=1)

print(x_descrete.shape)
x_descrete.head()


NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
WEEKDAY_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE
DEF_60_CNT_SOCIAL_CIRCLE
FLAG_DOCUMENT_2
FLAG_DOCUMENT_3
FLAG_DOCUMENT_4
FLAG_DOCUMENT_5
FLAG_DOCUMENT_6
FLAG_DOCUMENT_7
FLAG_DOCUMENT_8
FLAG_DOCUMENT_9
FLAG_DOCUMENT_10
FLAG_DOCUMENT_11
FLAG_DOCUMENT_12
FLAG_DOCUMENT_13
FLAG_DOCUMENT_14
FLAG_DOCUMENT_15
FLAG_DOCUMENT_16
FLAG_DOCUMENT_17
FLAG_DOCUMENT_18
FLAG_DOCUMENT_19
FLAG_DOCUMENT_20
FLAG_DOCUMENT_21
AMT_REQ_CREDIT_BUREAU_HOUR
AMT_REQ_CREDIT_BUREAU_DAY
AMT_REQ_CREDIT_BUREAU_WEEK
(307511, 166)


,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,...,AMT_REQ_CREDIT_BUREAU_DAY_9.0,AMT_REQ_CREDIT_BUREAU_WEEK_0.0,AMT_REQ_CREDIT_BUREAU_WEEK_1.0,AMT_REQ_CREDIT_BUREAU_WEEK_2.0,AMT_REQ_CREDIT_BUREAU_WEEK_3.0,AMT_REQ_CREDIT_BUREAU_WEEK_4.0,AMT_REQ_CREDIT_BUREAU_WEEK_5.0,AMT_REQ_CREDIT_BUREAU_WEEK_6.0,AMT_REQ_CREDIT_BUREAU_WEEK_7.0,AMT_REQ_CREDIT_BUREAU_WEEK_8.0
0,1,0,0,1,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,1,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


## continuous features
### replace NA with mean, normalize

In [29]:
from sklearn import preprocessing

train_number_features = train_data.select_dtypes(include='number').columns.values

x_continuous = pd.DataFrame()

continuous_features = []

for f in train_number_features:
    if train_discrete_features[f] == False:
        #print(f)
        
        no_na = train_data[f].fillna(train_data[f].mean())
        no_na = no_na.values.astype(float).reshape(-1, 1)

        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(no_na)
        df = pd.DataFrame(x_scaled, columns=[f])
        
        x_continuous = pd.concat([x_continuous, df],axis=1)
        
        continuous_features.append(f)
        cnt += 1

#x_continuous.head()

print(continuous_features)

['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'E

## combine X, y

In [43]:
from sklearn.model_selection import train_test_split

X = pd.concat([x_continuous, x_descrete],axis=1)
y = application_train.TARGET
#X.head()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(x_continuous.shape)
print(x_descrete.shape)
print(X.shape)

discrete_features_all = x_descrete.columns

(307511, 66)
(307511, 166)
(307511, 232)


## PCA

In [16]:
'''
from sklearn.decomposition import PCA

pca = PCA(n_components=10)

principalComponents = pca.fit_transform(X)

principalDf = pd.DataFrame(data = principalComponents)

X = principalDf

principalDf.head()
'''

'\nfrom sklearn.decomposition import PCA\n\npca = PCA(n_components=10)\n\nprincipalComponents = pca.fit_transform(X)\n\nprincipalDf = pd.DataFrame(data = principalComponents)\n\nX = principalDf\n\nprincipalDf.head()\n'

## Train

In [17]:
#---------------------------------------SVM -------------------------------------------------
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import accuracy_score

# Batch
def iter_minibatches(chunksize):
    
    # Provide chunks one by one
    chunkstartmarker = 0
    numtrainingpoints = 3000#307511
    
    while chunkstartmarker < numtrainingpoints:
        #chunkrows = range(chunkstartmarker,chunkstartmarker+chunksize)
        #X_chunk, y_chunk = getrows(chunkrows)
        start = chunkstartmarker
        end = chunkstartmarker+chunksize
        X_chunk = X_train.iloc[start:end]
        y_chunk = y_train.iloc[start:end]
        yield X_chunk, y_chunk, chunkstartmarker
        chunkstartmarker += chunksize
        
def train():
    batcherator = iter_minibatches(chunksize=1000)
    model = SGDRegressor()
 
    # Train model
    for X_chunk, y_chunk, chunkstartmarker in batcherator:
        model.partial_fit(X_chunk, y_chunk)
        #print(chunkstartmarker)
        print(model.score(X_test, y_test))
        print(model.predict(X_test))
    

    accu = model.score(X_test, y_test)
    print('accu = ', accu)
 
    # Now make predictions with trained model
    #y_predicted = model.predict(X_test)
    
train()

y.head()

c:\users\ian\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.016415846112421306
[0.11912888 0.11675758 0.05822231 ... 0.07704757 0.1038368  0.08239088]
0.021122521003099304
[ 0.09353602  0.02890983 -0.01925137 ...  0.0306441   0.05150805
  0.07331517]
0.03193365192766895
[0.11870987 0.04566393 0.01533807 ... 0.10329465 0.09055104 0.05598642]
accu =  0.03193365192766895


0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [18]:

from sklearn.linear_model import LogisticRegression

classifier =  LogisticRegression(verbose=True)

classifier.fit(X, y)
print(classifier.score(X_test, y_test))

[LibLinear]0.9176313735691988


In [20]:
from sklearn.externals import joblib

joblib.dump(classifier, 'save/clf_LogisticRegression.pkl')

classifier = joblib.load('save/clf_LogisticRegression.pkl')

print(classifier.predict(X[0:1]))

[1]


## output result

In [60]:
from sklearn.externals import joblib
#discrete_features
#continuous_features

def output():
    application_test = pd.read_csv('../input/application_test.csv')
    classifier = joblib.load('save/clf_LogisticRegression.pkl')
    
    # descrete
    
    test_descrete = pd.DataFrame()
    for f in discrete_features:
        onehot = pd.get_dummies(application_test[f])# this will loss some features with 0s
        onehot.columns = [f + '_' + str(col) for col in onehot.columns]
        test_descrete = pd.concat([test_descrete, onehot],axis=1)
    
    for f in discrete_features_all:# fill missing descrete features
        if f not in test_descrete.columns:
            test_descrete[f]= 0
    
    for f in test_descrete.columns:# remove features not in training data
        if f not in discrete_features_all:
            test_descrete = test_descrete.drop(columns=[f])
    
    # continuous
    
    test_continuous = pd.DataFrame()
    for f in continuous_features:
        no_na = application_test[f].fillna(train_data[f].mean())
        no_na = no_na.values.astype(float).reshape(-1, 1)

        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(no_na)
        df = pd.DataFrame(x_scaled, columns=[f])
        
        test_continuous = pd.concat([test_continuous, df],axis=1)
    

    test_data = pd.concat([test_continuous, test_descrete],axis=1)
    
    #print(test_continuous.shape)
    #print(test_descrete.shape)
    print(test_data.shape)
    #print(len(discrete_features))
    #print(test_descrete.columns)
    #print(x_descrete.columns)
    #print(len(discrete_features_all))
    
    p = classifier.predict(test_data)
    print(p)
    
    result = pd.DataFrame()
    result['SK_ID_CURR'] = application_test['SK_ID_CURR']
    result['TARGET'] = p
    print(result.head())
    
    result.to_csv('./output/out.csv')
    
output()

(48744, 232)
[0 0 0 ... 0 0 0]
   SK_ID_CURR  TARGET
0      100001       0
1      100005       0
2      100013       0
3      100028       0
4      100038       0
